In [1]:
%xmode Minimal

Exception reporting mode: Minimal


In [2]:
%%capture pipoutput
%pip install python-dotenv boto3 pandas pyarrow fastparquet 

In [3]:
%%capture pipoutput
%pip install --upgrade osc-ingest-tools

In [4]:
import uuid
#uniq = uuid.uuid4().hex[:8]
uniq = '23b38211'
uniq

'23b38211'

In [5]:
ingest_catalog = 'osc_datacommons_dev'
ingest_schema = 'demo'

In [6]:
partition_columns = ['year']

In [7]:
iceberg_catalog = 'osc_datacommons_iceberg_dev'
iceberg_schema = 'iceberg_demo'
iceberg_table = f'iceberg_{uniq}'

In [8]:
import osc_ingest_trino as osc
osc.load_credentials_dotenv()

engine = osc.attach_trino_engine(catalog = iceberg_catalog, verbose=True)

using connect string: trino://erikerlandson@trino-secure-odh-trino.apps.odh-cl1.apps.os-climate.org:443/osc_datacommons_iceberg_dev


In [9]:
# bucket must be configured with credentials for trino, and accessible to the hive catalog
bucket = osc.attach_s3_bucket('S3_DEV')

In [10]:
import random
import pandas as pd

datasize = 1000
yeardata = [random.randrange(2020,2023) for x in range(datasize)]
metricdata = [random.random() for x in range(datasize)]

df1 = pd.DataFrame(data={"year": yeardata, "metric": metricdata})
df1.convert_dtypes()

,year,metric
0,2022,0.152365
1,2021,0.352381
2,2021,0.655158
3,2020,0.413705
4,2022,0.871667
...,...,...
995,2022,0.618417
996,2021,0.582545
997,2021,0.553149
998,2020,0.708774


In [11]:
df1.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   year    1000 non-null   int64  
 1   metric  1000 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 15.8 KB


In [12]:
osc.fast_pandas_ingest_via_hive(
    df1,
    engine,
    iceberg_catalog, iceberg_schema, iceberg_table,
    bucket, ingest_catalog, ingest_schema,
    partition_columns = partition_columns,
    overwrite = False,
    verbose = False
)

In [13]:
sql = f"""
select * from {iceberg_catalog}.{iceberg_schema}.{iceberg_table}
"""
df = pd.read_sql(sql, engine).convert_dtypes()
df

,metric,year
0,0.906234,2021
1,0.093459,2021
2,0.744767,2021
3,0.065963,2021
4,0.286204,2020
...,...,...
2995,0.983955,2022
2996,0.797005,2022
2997,0.767044,2022
2998,0.805545,2022
